In [1]:
#ETL 


#LEER ARCHIVOS JSON 

#funciones 

import ast
import pandas as pd

def leer_json(ruta_archivo):
    """
    Carga un archivo JSON y devuelve un DataFrame.

    Parameters:
        ruta_archivo (str): La ruta al archivo JSON.

    Returns:
        pd.DataFrame: El DataFrame cargado desde el archivo JSON.
    """
    filas = []

    # Abrir el archivo en modo lectura con la codificación "MacRoman"
    with open(ruta_archivo, encoding="MacRoman") as archivo:
        # Iterar sobre cada línea en el archivo
        for linea in archivo.readlines():
            # Evaluar la línea como una expresión de Python y agregarla a la lista
            filas.append(ast.literal_eval(linea))

    # Crear y devolver un DataFrame a partir de la lista de filas
    return pd.DataFrame(filas)


def desanidar_columna(df_entrada, columna_anidada):
    """
    Expande una columna anidada en un DataFrame.

    Parameters:
    - df_entrada (pd.DataFrame): DataFrame de entrada con la columna anidada.
    - columna_anidada (str): Nombre de la columna anidada que se va a expandir.

    Returns:
    pd.DataFrame: Nuevo DataFrame con la columna anidada expandida.
    """
    # Inicializar una lista vacía para almacenar los registros expandidos.
    registros_expandidos = []

    # Iterar sobre cada fila del DataFrame de entrada.
    for indice, fila in df_entrada.iterrows():
        # Extraer los datos base de la fila eliminando la columna anidada y convirtiéndolos a un diccionario.
        datos_base = fila.drop(columna_anidada).to_dict()
        # Obtener la lista de datos anidados de la columna especificada.
        datos_anidados_lista = fila[columna_anidada]

        # Iterar sobre cada conjunto de datos anidados.
        for datos_anidados in datos_anidados_lista:
            # Crear un nuevo registro combinando los datos base y los datos anidados.
            nuevo_registro = {**datos_base, **datos_anidados}
            # Agregar el nuevo registro a la lista de registros_expandidos.
            registros_expandidos.append(nuevo_registro)

    # Crear un nuevo DataFrame a partir de la lista de registros_expandidos y devolverlo.
    df_expandido = pd.DataFrame(registros_expandidos)
    return df_expandido



In [3]:
#leer archivo items

# Ruta del archivo JSON
ruta_json ='australian_users_items.json'

# Cargar el archivo JSON en un DataFrame
df_user_items = leer_json(ruta_json)

# Expandir los diccionarios en varias filas
df_user_items = desanidar_columna(df_user_items, "items")
df_user_items.head()

FileNotFoundError: [Errno 2] No such file or directory: 'australian_users_items.json'

In [5]:
#ver valores nulos 
df_user_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [6]:
#revisar duplicados
df_user_items.drop_duplicates(inplace=True)
df_user_items.shape

(5094105, 8)

In [7]:
#eliminar columnas innecesarias
df_user_items= df_user_items.drop(['steam_id', 'playtime_2weeks'], axis=1)

In [8]:
#Guardamos el dataframe en formato csv para su posterior uso

df_user_items.to_parquet('user_items.parquet', index=False)

In [9]:
#leer archivo reviews

# Ruta del archivo JSON
ruta_json ='australian_user_reviews.json'

# Cargar el archivo JSON en un DataFrame
df_user_reviews = leer_json(ruta_json)

# Expandir los diccionarios en varias filas
df_user_reviews = desanidar_columna(df_user_reviews, "reviews")
df_user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [10]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


In [11]:
##ver nulos


df_user_reviews.isnull().sum()

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

In [12]:
#ver duplicados 

#revisar y eliminar  duplicados
df_user_reviews.drop_duplicates(inplace=True)
df_user_reviews.shape

(58431, 9)

In [13]:
# extraer el año de columna Posted

#extraer año de posteo 

df_user_reviews['posted'] =df_user_reviews['posted'].str.replace(r'.*?(\d{4}).*', r'\1', regex=True)

# Ahora, la columna 'posted' contiene solo el año

print(df_user_reviews['posted'])

0                   2011
1                   2011
2                   2011
3                   2014
4                   2013
              ...       
59300    Posted July 10.
59301     Posted July 8.
59302     Posted July 3.
59303    Posted July 20.
59304     Posted July 2.
Name: posted, Length: 58431, dtype: object


In [14]:
import re
# como se puede visualizar hay registros que no contienen el año de posteo por lo que no se  han transformado y reemplazarlos por el valor 0

# Definir una expresión regular para verificar si un valor es un año numérico
regex_pattern = r'^\d{4}$'

# Marcar los valores que no son años numéricos con 0
df_user_reviews['posted'] = df_user_reviews['posted'].apply(lambda x: 0 if not str(x).strip().replace('.', '').isdigit() and not bool(re.match(regex_pattern, str(x))) else x)

# Verificar el contenido actual de la columna 'posted'
print(df_user_reviews['posted'])

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
59300       0
59301       0
59302       0
59303       0
59304       0
Name: posted, Length: 58431, dtype: object


In [15]:
##eliminar filas con año 0 ya que no nos serviria para nuestra consulta posterior

# Eliminar filas donde el año es igual a 0
df_user_reviews= df_user_reviews[df_user_reviews['posted'] != 0]
print(df_user_reviews['posted'])

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
59252    2015
59255    2015
59265    2015
59267    2015
59276    2015
Name: posted, Length: 48498, dtype: object


In [16]:
#eliminar filas sin reviews
print(df_user_reviews['review'])

0        Simple yet with great replayability. In my opi...
1                     It's unique and worth a playthrough.
2        Great atmosphere. The gunplay can be a bit chu...
3        I know what you think when you see this title ...
4        For a simple (it's actually not all that simpl...
                               ...                        
59252                                       its FUNNNNNNNN
59255    Awesome fantasy game if you don't mind the gra...
59265                                     Prettyy Mad Game
59267                                   AMAZING GAME 10/10
59276    Why I voted yes? 1. Girl characters have boobs...
Name: review, Length: 48498, dtype: object


In [17]:
# Contar cuántas filas tienen texto vacío en la columna 'Review'
filas_con_texto_vacio = df_user_reviews['review'].str.strip().eq('')
cantidad_filas_vacias = filas_con_texto_vacio.sum()

# Mostrar la cantidad de filas con texto vacío en la columna 'Review'
print("\nCantidad de filas con texto vacío en la columna 'review':", cantidad_filas_vacias)


Cantidad de filas con texto vacío en la columna 'review': 35


In [18]:
# Eliminar las filas que contienen texto vacío en la columna 'Review'
df_user_reviews_final= df_user_reviews[~filas_con_texto_vacio]

# Mostrar el DataFrame después de eliminar las filas con texto vacío en la columna 'Review'

print(df_user_reviews_final.head(2))

             user_id                                           user_url funny   
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...        \
1  76561197970982479  http://steamcommunity.com/profiles/76561197970...         

  posted last_edited item_id         helpful  recommend   
0   2011                1250  No ratings yet       True  \
1   2011               22200  No ratings yet       True   

                                              review  
0  Simple yet with great replayability. In my opi...  
1               It's unique and worth a playthrough.  


In [19]:
#eliminamos columnas innecesarias 

df_user_reviews_final.drop(columns=['user_url', 'funny', 'last_edited', 'helpful'], axis=1,inplace=True)

C:\Users\CIN\AppData\Local\Temp\ipykernel_2948\2360318052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews_final.drop(columns=['user_url', 'funny', 'last_edited', 'helpful'], axis=1,inplace=True)


In [20]:
df_user_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48463 entries, 0 to 59276
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    48463 non-null  object
 1   posted     48463 non-null  object
 2   item_id    48463 non-null  object
 3   recommend  48463 non-null  bool  
 4   review     48463 non-null  object
dtypes: bool(1), object(4)
memory usage: 1.9+ MB


In [21]:
#convertimos a csv
df_user_reviews_final.to_csv('user_reviews.csv', index=False)

In [22]:
#Leemos el archivo output_steam_games
import pandas as pd
import json


steams_games=[]
with open('output_steam_games.json', 'r') as file:
    for line in file:
        k = json.loads(line)
        steams_games.append(k)


df_steam_games = pd.DataFrame(steams_games)

df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [23]:
##verificamos cuantos registros nulos hay
num_nulos_por_columna = df_steam_games.isnull().sum()
print(num_nulos_por_columna)

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64


In [24]:
# Calcula el número de filas con nulos en todas sus columnas
num_filas_con_nulos_en_todas_las_columnas = df_steam_games.isnull().all(axis=1).sum()

# Muestra el número de filas con nulos en todas sus columnas
print("Número de filas con nulos en todas sus columnas:", num_filas_con_nulos_en_todas_las_columnas)

Número de filas con nulos en todas sus columnas: 88310


In [25]:
# Elimina filas y columnas que contienen valores nulos y actualiza el DataFrame
df_steam_games_limpio = df_steam_games.dropna(axis=0, how='any').dropna(axis=1, how='any')

# Muestra información sobre el DataFrame después de eliminar filas y columnas nulas
print("Información del DataFrame después de eliminar filas y columnas nulas:")
print(df_steam_games_limpio.info())
df_steam_games_limpio.head()

Información del DataFrame después de eliminar filas y columnas nulas:
<class 'pandas.core.frame.DataFrame'>
Index: 22530 entries, 88310 to 120443
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22530 non-null  object
 1   genres        22530 non-null  object
 2   app_name      22530 non-null  object
 3   title         22530 non-null  object
 4   url           22530 non-null  object
 5   release_date  22530 non-null  object
 6   tags          22530 non-null  object
 7   reviews_url   22530 non-null  object
 8   specs         22530 non-null  object
 9   price         22530 non-null  object
 10  early_access  22530 non-null  object
 11  id            22530 non-null  object
 12  developer     22530 non-null  object
dtypes: object(13)
memory usage: 2.4+ MB
None


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd


In [26]:
# Reiniciamos los índices del DataFrame 
df_steam_games_limpio = df_steam_games_limpio.reset_index(drop=True)

df_steam_games_limpio.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd


In [35]:
# Eliminamos columnas innecesarias 

columnas_a_eliminar=['url','publisher','tags','reviews_url','specs','early_access']
steam_games_final=df_steam_games_limpio.drop(columns=columnas_a_eliminar)

In [36]:
print(steam_games_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22530 entries, 0 to 22529
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        22530 non-null  object
 1   app_name      22530 non-null  object
 2   title         22530 non-null  object
 3   release_date  22530 non-null  object
 4   price         22530 non-null  object
 5   id            22530 non-null  object
 6   developer     22530 non-null  object
dtypes: object(7)
memory usage: 1.2+ MB
None


In [37]:
#Renombrar Columnas: decidí colocar todas los titulos de las columnas en español para que sea mas amigable al momento de usar la api

titulos_nuevos={'genres':'genero','title':'titulo','release_date':'anio_lanzamiento','price':'precio','id':'id_contenido','developer':'desarrollador'}
df_steam_nuevo=steam_games_final.rename(columns=titulos_nuevos)

df_steam_nuevo.head()

,genero,app_name,titulo,anio_lanzamiento,precio,id_contenido,desarrollador
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd


In [38]:
#modificamos  columna anio_lanzamiento para  dejar solo el añio
# Convertimos  la columna 'fecha' a formato datetime, intentando manejar diferentes formatos
df_steam_nuevo['anio_lanzamiento'] = pd.to_datetime(df_steam_nuevo['anio_lanzamiento'],errors='coerce')

# Extraemos el anio convirtiendo la serie resultante a formato de texto
df_steam_nuevo['anio_lanzamiento'] = df_steam_nuevo['anio_lanzamiento'].dt.strftime('%Y')

# Muestra el DataFrame después de modificar la columna 'fecha'
df_steam_nuevo.head()


,genero,app_name,titulo,anio_lanzamiento,precio,id_contenido,desarrollador
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018,Free To Play,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017,Free to Play,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017,0.99,767400,彼岸领域
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018,3.99,772540,Trickjump Games Ltd


In [39]:
## se verificaron que en la columna precio había otros formatos como ejemplo  Free to Use, Free to Play] por 0.00

df_steam_nuevo['precio']=pd.to_numeric(df_steam_nuevo['precio'],errors='coerce')
# Reemplazar los valores no numéricos por 0.00
df_steam_nuevo['precio'].fillna(0.00, inplace=True)
print(df_steam_nuevo['precio'])
print()

0        4.99
1        0.00
2        0.00
3        0.99
4        3.99
         ... 
22525    1.99
22526    1.99
22527    4.99
22528    1.99
22529    4.99
Name: precio, Length: 22530, dtype: float64



In [40]:
## revision de duplicados 

# Verificar duplicados en la columna 'id'
duplicates_id = df_steam_nuevo.duplicated(subset=['id_contenido'], keep=False)

# Filtrar el DataFrame para mostrar solo las filas duplicadas en 'id'
duplicated_rows = df_steam_nuevo[duplicates_id]

print(duplicated_rows)


         genero                          app_name   
9654   [Action]  Wolfenstein II: The New Colossus  \
10102  [Action]  Wolfenstein II: The New Colossus   

                                 titulo anio_lanzamiento  precio id_contenido   
9654   Wolfenstein II: The New Colossus             2017   59.99       612880  \
10102  Wolfenstein II: The New Colossus             2017   59.99       612880   

       desarrollador  
9654   Machine Games  
10102  Machine Games  


In [41]:
# Eliminar filas duplicadas en la columna 'id'
df_sin_duplicados = df_steam_nuevo.drop_duplicates(subset=['id_contenido'])

print(df_sin_duplicados)

                                                  genero   
0          [Action, Casual, Indie, Simulation, Strategy]  \
1                   [Free to Play, Indie, RPG, Strategy]   
2      [Casual, Free to Play, Indie, Simulation, Sports]   
3                            [Action, Adventure, Casual]   
4                        [Action, Adventure, Simulation]   
...                                                  ...   
22525                 [Action, Adventure, Casual, Indie]   
22526              [Casual, Indie, Simulation, Strategy]   
22527                          [Casual, Indie, Strategy]   
22528                        [Indie, Racing, Simulation]   
22529                                    [Casual, Indie]   

                       app_name                    titulo anio_lanzamiento   
0           Lost Summoner Kitty       Lost Summoner Kitty             2018  \
1                     Ironbound                 Ironbound             2018   
2       Real Pool 3D - Poolians   Real Pool 3

In [42]:
#Guardamos el dataframe en formato csv para su posterior uso

df_sin_duplicados.to_csv('steam_games.csv', index=False)